In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


## 设定网络参数

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10
n_sample = mnist.train.num_examples

## 定义Placeholder 占位符

In [5]:
with tf.name_scope('inputs'):
    x = tf.placeholder('float', [None, n_input], name = 'input_x')
    y = tf.placeholder('float', [None, n_classes], name = 'input_y')

In [6]:
with tf.name_scope('input_reshape'):
    image_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_input, 10)

## 建构多层神经网络

In [7]:
def add_layer(x, input_tensors, output_tensors, layer_name, activation_function = None):
    with tf.name_scope('Layer'):
        with tf.name_scope('Weights'):
            weight = tf.Variable(tf.random_normal([input_tensors, output_tensors]), name = 'w')
            tf.summary.histogram(name = layer_name + '/Weights', values = weight)
        with tf.name_scope('Bias'):
            bias = tf.Variable(tf.random_normal([output_tensors]), name = 'b')
            tf.summary.histogram(name = layer_name + '/Bias', values = bias)
        with tf.name_scope('Wx_plus_b'):
            formula = tf.add(tf.matmul(x, weight), bias)
        if activation_function is None:
            outputs = formula
        else:
            outputs = activation_function(formula)
        tf.summary.histogram(name = layer_name + '/Outputs', values = outputs)
        return outputs

## 添加隐藏层

In [8]:
layer1 = add_layer(x, input_tensors=n_input, output_tensors=n_hidden_1, layer_name='layer1', activation_function=tf.nn.relu)
layer2 = add_layer(layer1, input_tensors=n_hidden_1, output_tensors=n_hidden_2, layer_name='layer2', activation_function=tf.nn.relu)
out_layer = add_layer(layer2, input_tensors=n_hidden_2, output_tensors=n_classes, layer_name='out_layer', activation_function=None)

## 设定代价函数

In [9]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=y))
    tf.summary.scalar('loss', cost)
    
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    tf.summary.scalar('accuracy', acc)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## 初始化所有变量

In [10]:
init = tf.global_variables_initializer()

# 训练模型

## 执行会话

In [11]:
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    
    ## Merge Summary
    writer = tf.summary.FileWriter('tensorboard/', graph = sess.graph)
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(n_sample/batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c, result = sess.run([optimizer, cost, merged], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
            
            ## Adding summary of each step
            writer.add_summary(result, epoch * total_batch + i)
            
        print('Epoch: {} cost = {}'.format(epoch+1, avg_cost))
        
    print('Training Completed in {} Epochs'.format(training_epochs))

Epoch: 1 cost = 144.93075491818522
Epoch: 2 cost = 55.71438606088812
Epoch: 3 cost = 42.00464901360596
Epoch: 4 cost = 34.27296849337493
Epoch: 5 cost = 29.229890615073128
Epoch: 6 cost = 25.58724262410944
Epoch: 7 cost = 22.831245736642312
Epoch: 8 cost = 20.65179184198379
Epoch: 9 cost = 18.839310496286913
Epoch: 10 cost = 17.338353062488814
Epoch: 11 cost = 16.005965529355123
Epoch: 12 cost = 14.902288224046883
Epoch: 13 cost = 13.878019954833123
Epoch: 14 cost = 13.045792704007846
Epoch: 15 cost = 12.291746347384032
Training Completed in 15 Epochs
